# BUG: Beveridgean Unemployment Gap
This series of notebooks demonstrate the python implementation of the "Beveridgean Unemployment Gap" by Pascal Michaillat and Emmanuel Saez. The original code was in MATLAB. See [GitHub link](https://github.com/pascalmichaillat/unemployment-gap) for original. 

## Appendix
This notebook examines the python implementation of functions in the paper's online appendix. [link](https://ars.els-cdn.com/content/image/1-s2.0-S2666551421000048-mmc1.pdf)

## import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
import sys
sys.path.insert(0, '../bug')
import bug

## Read the data
Here, we read from the [excel file](https://github.com/pascalmichaillat/unemployment-gap/blob/main/code/data.xlsx) provided with the unemployment-gap matlab package.

The goal of this notebook is to re-create function and figures from the Unemployment Gap paper appendix, so that we can verify we are getting the *same* outputs. (*Sameness* allowing for some small differences between the two language implementations)

#### unemployment rate

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment rate (percent)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_rate = pd.Series(data=df['Unemployment rate (percent)'].values,
                       index=dates, name='unempl_rate')

#### unemployment level

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Unemployment level (thousands of persons)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
unempl_level = pd.Series(data=df['Unemployment level (thousands of persons)'].values,
                        index=dates, name='unempl_level')

#### short-term unemployment level

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Short-term unemployment level (thousands of persons)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
ushort_level = pd.Series(data=df['Short-term unemployment level (thousands of persons)'].values, 
                        index=dates, name='short_term_unempl_level')

#### labor force level

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Monthly data',
                           header=1, usecols=['Labor force level (thousands of persons)', 'Year', 'Month'],)

In [ ]:
# set the index 
dates = pd.PeriodIndex(pd.to_datetime(dict(year=df.Year, month=df.Month, day=15)).dt.to_period('m') ) 
labor_level = pd.Series(data=df['Labor force level (thousands of persons)'].values,
                       index=dates, name='labor_force_level')

#### Recession information

In [ ]:
df = pd.read_excel('../../code/data.xlsx', sheet_name='Recession dates',header=1, 
                   usecols=['Peak month', 'Trough month'],).drop([0]).reset_index()
starts =  pd.to_datetime(df['Peak month'])

ends = pd.to_datetime(df['Trough month'])

# Appendix B: 
## Job-finding and job-separation rates

### Job-finding rate
Call the function `bug.compute_job_finding_rate()` with arguments:
  * unemployment level, required
  * short-term unemployment level, required
  * True/False for whether to convert monthly to quarterly, optional; default is True
  * True/False for whether to adjust the short-term level after 1994, as in Shimer (2012), optional; default is True
  

In [ ]:
find_rate_q =  bug.compute_job_finding_rate(unempl_level, ushort_level, True, True)
find_rate_m =  bug.compute_job_finding_rate(unempl_level, ushort_level, False, True)

#### Re-create figure A1
Job-finding rate in the US, 1951-2019

In [ ]:
ax = find_rate_q[:-1].plot(figsize=(9,7), linewidth=1.5, label='Quarterly (python output)')
find_rate_m[:-1].plot(ax=ax, linewidth=1.5, label='Monthly (python output)')


plt.ylim(0, find_rate_q.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends],)

plt.ylabel('Job-finding rate', fontsize=13)
handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12)
plt.title('Python Outputs', fontsize=14)

In [ ]:
fig_A1 = pd.read_excel('../../figures/xlsx/figureA1.xlsx', header=1, usecols=['Quarterly job-finding rate']) 
fig_A1.set_index(find_rate_q.index[:-1], inplace=True)

In [ ]:
find_rate_q =  bug.compute_job_finding_rate(unempl_level, ushort_level, True, True)

In [ ]:
ax = fig_A1.rename(columns={'Quarterly job-finding rate': 'Quarterly (matlab output)'}).plot(figsize=(9,7), 
                linewidth=3, color='k', linestyle='dotted',)
find_rate_q[:-1].plot(ax=ax, linewidth=1.5, label='Quarterly (python output)')

plt.ylim(0, find_rate_q.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends])

plt.ylabel('Job-finding rate', fontsize=13)
handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12)
plt.title('Python vs MATLAB Outputs', fontsize=14)

In [ ]:
plt.scatter(x=fig_A1, y=find_rate_q[:-1])
plt.title('Python vs MATLAB Outputs', fontsize=14)

### Job-separation rate
Call the function `bug.compute_job_separation_rate()` with arguments:
  * unemployment level, required
  * short-term unemployment level, required
  * labor force level, required
  * True/False for whether to convert monthly to quarterly, optional; default is True
  * True/False for whether to adjust the short-term level after 1994, as in Shimer (2012), optional; default is True


In [ ]:
sep_rate_q = bug.compute_job_separation_rate(unempl_level, ushort_level, labor_level, True, True)
sep_rate_m = bug.compute_job_separation_rate(unempl_level, ushort_level, labor_level, False, True)

#### Re-create figure A2
Job-separation rate in the US, 1951-2019

In [ ]:
ax = sep_rate_q[:-1].plot(figsize=(9,7), linewidth=1.5, label='Quarterly (python output)')
sep_rate_m[:-1].plot(ax=ax, linewidth=1.5, label='Monthly (python output)')

plt.ylim(0, sep_rate_q.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends])
handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12)

plt.ylabel('Job-separation rate', fontsize=13)
plt.title('Python Outputs', fontsize=14)

In [ ]:
fig_A2 = pd.read_excel('../../figures/xlsx/figureA2.xlsx', header=1, usecols=['Quarterly job-separation rate']) 
fig_A2.set_index(sep_rate_q.index[:-1], inplace=True)

In [ ]:
sep_rate_q = bug.compute_job_separation_rate(unempl_level, ushort_level, labor_level, True,True)

In [ ]:
ax = fig_A2.rename(columns={'Quarterly job-separation rate': 'Quarterly (matlab output)'}).plot(figsize=(9,7), 
                                    linewidth=3, linestyle='dotted', color='k', alpha=.8 )
sep_rate_q[:-1].plot(ax=ax, linewidth=1.5, label='Quarterly (python output)')

plt.ylim(0, sep_rate_q.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends])

plt.ylabel('Job-separation rate', fontsize=13)
plt.title('Python vs MATLAB Outputs', fontsize=14)

handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12)

In [ ]:
plt.scatter(x=fig_A2, y=sep_rate_q[:-1])
plt.title('Python vs MATLAB Outputs', fontsize=14)

#### Re-create figure A3
Beveridgean unemployment rate vs. actual unemployment

In [ ]:
bev_unempl = bug.compute_beveridgean_unemployment(find_rate_q, sep_rate_q)

In [ ]:
bev_unempl.head()

In [ ]:
unempl_q = unempl_rate.resample('Q').mean()/100.

In [ ]:
ax = unempl_q.plot(figsize=(9,7), linewidth=3, color='m', alpha=.8,
                    label='actual unemployemnt', linestyle='dotted')
bev_unempl.plot(ax=ax, linewidth=1.5, label='beveridgean (python output)')

plt.ylim(0, unempl_q.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends])

plt.ylabel('Unemployment rate', fontsize=13)
plt.title('Python Output vs Actual (quarterly)', fontsize=14)

handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12, loc=4)

In [ ]:
fig_A3 = pd.read_excel('../../figures/xlsx/figureA3.xlsx', header=1, 
                       usecols=['Beveridgean unemployment rate']) 
fig_A3.set_index(sep_rate_q.index[:-1], inplace=True)

In [ ]:
ax = fig_A3.rename(columns={'Beveridgean unemployment rate': 'Beveridgean (matlab output)'}).plot(figsize=(9,7), 
                linewidth=3, color='k', linestyle='dotted')
bev_unempl[:-1].plot(ax=ax, linewidth=1.5, label='Beveridgean (python output)')

plt.ylim(0, unempl_q.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends])

plt.ylabel('unemployement rate', fontsize=13)
plt.title('Python vs MATLAB Outputs', fontsize=14)

handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12, loc=4)

In [ ]:
plt.scatter(x=fig_A3, y=bev_unempl[:-1])
plt.title('Python vs MATLAB Outputs', fontsize=14)

In [ ]:
bev_unempl_m = bug.compute_beveridgean_unemployment(find_rate_m, sep_rate_m)

In [ ]:
unempl_m = unempl_rate/100.

In [ ]:

ax = bev_unempl_m[:-1].plot(linewidth=3, figsize=(9,7),label='beveridgean (python output)')
ax = unempl_m[1:].plot( ax=ax, linewidth=2, color='m', 
                     label='actual unemployemnt')

plt.ylim(0, unempl_m.max()*1.1)
bug.format_plot(ax, recession_dates=[starts, ends])

plt.ylabel('Unemployment rate', fontsize=13)
plt.title('Python Output vs Actual (monthly)', fontsize=14)

handles, labels = ax.get_legend_handles_labels()
handles.append(Patch(facecolor='grey',))
labels.append("Recession indicator")
plt.legend(handles=handles, labels=labels,fontsize=12, loc=4)